# Generate ConnectivityFiles for HealPix grids to be used with TempestExtremes

By Bryce Harrop

Adapted by Stella Bourdin

In [16]:
import uxarray as ux
import xarray as xr
import warnings
import os

Let's generate exodus grid files for healpix grids at varous zoom levels using uxarray.  The end goal is to generate connectivity files to be used with Tempest Extremes.  Testing has shown that this connectivity file is identical regardless of the choice for `pixels_only`, so it will be set to `False` by default.

At the time of writing, the exodus files do not have a necessary dimension, `num_elem`, that provides the total number of elements.  It is anticipated that this will be added by the uxarray team, so the call to `add_num_elem` can be skipped in the future.

In [17]:
# User settings
zoom_levels      = [0,1] #[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10]
pixels_only      = False
grid_format      = 'exodus'
do_add_num_elem  = True
scr_dir = '/work/scratch-nopw2/sbourdin/ConnectivityFiles/'

In [18]:
def add_num_elem(ds_grid):
    if 'num_elem' in ds_grid.dims:
        warnings.warn('num_elem is already a dimension.  Exiting add_num_elem()')
    else:
        print('adding in dimension "num_elem"')
        # Assuming you have the number of elements from the block; num_el_in_blk1
        num_elem = 0
        for dim in ds_grid.dims:
            if 'num_el_in_blk' in dim:
                num_elem = num_elem + ds_grid[dim].size
                print(dim, ds_grid[dim].size, num_elem)
        print('Total num_elem is ', num_elem)

        # Add the new dimension 'num_elem' to the ds_grid
        ds_grid = ds_grid.assign_coords(num_elem=range(num_elem))

        # Add a new variable using this dimension
        ds_grid['num_elem'] = xr.DataArray(range(num_elem), dims='num_elem')

        # Add in metadata
        ds_grid['num_elem'].attrs['long_name'] = 'Total Number of Elements'
    return ds_grid

In [19]:
for zoom in zoom_levels:
    filename = 'healpix_grid_zoom_' + str(zoom) + '_format_' + grid_format + '.nc'
    print(filename)
    if os.path.exists(scr_dir + filename):
        print('output for zoom level ' + str(zoom) + ' already exists.  Skipping...')
    else:
        uxgrid_healpix = ux.Grid.from_healpix(zoom, pixels_only=pixels_only)
        ds_grid = uxgrid_healpix.to_xarray(grid_format=grid_format)
        if do_add_num_elem:
            ds_grid = add_num_elem(ds_grid)
        ds_grid.to_netcdf(scr_dir + filename, mode='w')
    print( )

healpix_grid_zoom_0_format_exodus.nc
output for zoom level 0 already exists.  Skipping...

healpix_grid_zoom_1_format_exodus.nc
output for zoom level 1 already exists.  Skipping...



# Break for running `generate_connectivity_files.sh`

Uh oh... the connectivity files made with the exodus grid files aren't working correctly.  The values for grid cell lon, lat, and area (the first three columns in the connectivity file) are incorrect and this causes Tempest Extremes' great-circle calculations to get messed up.

Let's run this thing again, but this time we will make the script files.

In [ ]:
grid_format      = 'scrip'
do_add_num_elem  = False

In [ ]:
for zoom in zoom_levels[-1:]:
    filename = 'healpix_grid_zoom_' + str(zoom) + '_format_' + grid_format + '.nc'
    print(filename)
    if os.path.exists(scr_dir + filename):
        print('output for zoom level ' + str(zoom) + ' already exists.  Skipping...')
    else:
        uxgrid_healpix = ux.Grid.from_healpix(zoom, pixels_only=pixels_only)
        ds_grid = uxgrid_healpix.to_xarray(grid_format=grid_format)
        if do_add_num_elem:
            ds_grid = add_num_elem(ds_grid)
        ds_grid.to_netcdf(scr_dir + filename, mode='w')
    print( )

In [ ]:
def correct_connectivity_file(scrip_file, broken_connect, updated_connect):

    # Load correct values from the NetCDF file using xarray
    correct_dataset = xr.open_dataset(scrip_file)

    # Assume correct_dataset has the correct longitude, latitude, and area
    correct_longitudes = correct_dataset['grid_center_lon'].values
    correct_latitudes  = correct_dataset['grid_center_lat'].values
    correct_areas      = correct_dataset['grid_area'].values

    # Read and parse your text file, skipping the second line
    with open(broken_connect, 'r') as f:
        header = f.readline().strip()
        metadata_line = f.readline().strip()
        data_lines = [line.strip() for line in f.readlines()]

    # Prepare updated lines
    updated_lines = []

    for i, line in enumerate(data_lines):
        values = line.split(',')
        # Overwrite incorrect values with correct ones
        values[0] = f"{correct_longitudes[i]:.14e}"
        values[1] = f"{correct_latitudes[i]:.14e}"
        values[2] = f"{correct_areas[i]:.14e}"
        updated_lines.append(','.join(values))

    # Write updated data back to a new text file
    if not os.path.exists(updated_connect):
        with open(updated_connect, 'w') as f:
            f.write(header + '\n')
            f.write(metadata_line + '\n')
            for line in updated_lines:
                f.write(line + '\n')
    else:
        print('Updated connect file already exists.')

    # Close the dataset
    correct_dataset.close()

    print("File updated successfully.")

In [ ]:
# Paths to your files
grid_path        = '../../ConnectivityFiles/'

for zoom in range(11):
    scrip_file       = scr_dir + 'healpix_grid_zoom_' + str(zoom) + '_format_scrip.nc'
    broken_connect   = scr_dir + 'connect_healpix_zoom_'+str(zoom)+'.txt'
    updated_connect  = grid_path + 'ConnectivityFiles_for_healpix_zoom_' + str(zoom) + '.txt'

    correct_connectivity_file(scrip_file, broken_connect, updated_connect)